In [ ]:
%%writefile genetic.py
import random
import statistics
import sys
import time
from bisect import bisect_left
from enum import Enum
from math import exp
 
 
def _generate_parent(length, geneSet, get_fitness):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness, Strategies.Create)
 
 
def _mutate(parent, geneSet, get_fitness):
    childGenes = parent.Genes[:]
    index = random.randrange(0, len(parent.Genes))
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate if newGene == childGenes[index] else newGene
    fitness = get_fitness(childGenes)
    return Chromosome(childGenes, fitness, Strategies.Mutate)
 
 
def _mutate_custom(parent, custom_mutate, get_fitness):
    childGenes = parent.Genes[:]
    custom_mutate(childGenes)
    fitness = get_fitness(childGenes)
    return Chromosome(childGenes, fitness, Strategies.Mutate)
 
 
def _crossover(parentGenes, index, parents, get_fitness, crossover, mutate,
               generate_parent):
    donorIndex = random.randrange(0, len(parents))
    if donorIndex == index:
        donorIndex = (donorIndex + 1) % len(parents)
    childGenes = crossover(parentGenes, parents[donorIndex].Genes)
    if childGenes is None:
        # parent and donor are indistinguishable
        parents[donorIndex] = generate_parent()
        return mutate(parents[index])
    fitness = get_fitness(childGenes)
    return Chromosome(childGenes, fitness, Strategies.Crossover)
 
 
def get_best(get_fitness, targetLen, optimalFitness, geneSet, display,
             custom_mutate=None, custom_create=None, maxAge=None, poolSize=1,
             crossover=None, maxSeconds=None):
    if custom_mutate is None:
        def fnMutate(parent):
            return _mutate(parent, geneSet, get_fitness)
    else:
        def fnMutate(parent):
            return _mutate_custom(parent, custom_mutate, get_fitness)
 
    if custom_create is None:
        def fnGenerateParent():
            return _generate_parent(targetLen, geneSet, get_fitness)
    else:
        def fnGenerateParent():
            genes = custom_create()
            return Chromosome(genes, get_fitness(genes), Strategies.Create)
 
    strategyLookup = {
        Strategies.Create: lambda p, i, o: fnGenerateParent(),
        Strategies.Mutate: lambda p, i, o: fnMutate(p),
        Strategies.Crossover: lambda p, i, o:
        _crossover(p.Genes, i, o, get_fitness, crossover, fnMutate,
                   fnGenerateParent)
    }
 
    usedStrategies = [strategyLookup[Strategies.Mutate]]
    if crossover is not None:
        usedStrategies.append(strategyLookup[Strategies.Crossover])
 
        def fnNewChild(parent, index, parents):
            return random.choice(usedStrategies)(parent, index, parents)
    else:
        def fnNewChild(parent, index, parents):
            return fnMutate(parent)
 
    for timedOut, improvement in \
            _get_improvement(fnNewChild, fnGenerateParent, maxAge, poolSize,
                             maxSeconds):
        if timedOut:
            return improvement
        display(improvement)
        f = strategyLookup[improvement.Strategy]
        usedStrategies.append(f)
        if not optimalFitness > improvement.Fitness:
            return improvement
 
 
def _get_improvement(new_child, generate_parent, maxAge, poolSize, maxSeconds):
    startTime = time.time()
    bestParent = generate_parent()
    yield maxSeconds is not None and time.time() \
          - startTime > maxSeconds, bestParent
    parents = [bestParent]
    historicalFitnesses = [bestParent.Fitness]
    for _ in range(poolSize - 1):
        parent = generate_parent()
        if maxSeconds is not None and time.time() - startTime > maxSeconds:
            yield True, parent
        if parent.Fitness > bestParent.Fitness:
            yield False, parent
            bestParent = parent
            historicalFitnesses.append(parent.Fitness)
        parents.append(parent)
    lastParentIndex = poolSize - 1
    pindex = 1
    while True:
        if maxSeconds is not None and time.time() - startTime > maxSeconds:
            yield True, bestParent
        pindex = pindex - 1 if pindex > 0 else lastParentIndex
        parent = parents[pindex]
        child = new_child(parent, pindex, parents)
        if parent.Fitness > child.Fitness:
            if maxAge is None:
                continue
            parent.Age += 1
            if maxAge > parent.Age:
                continue
            index = bisect_left(historicalFitnesses, child.Fitness, 0,
                                len(historicalFitnesses))
            proportionSimilar = index / len(historicalFitnesses)
            if random.random() < exp(-proportionSimilar):
                parents[pindex] = child
                continue
            bestParent.Age = 0
            parents[pindex] = bestParent
            continue
        if not child.Fitness > parent.Fitness:
            # same fitness
            child.Age = parent.Age + 1
            parents[pindex] = child
            continue
        child.Age = 0
        parents[pindex] = child
        if child.Fitness > bestParent.Fitness:
            bestParent = child
            yield False, bestParent
            historicalFitnesses.append(bestParent.Fitness)
 
 
class Chromosome:
    def __init__(self, genes, fitness, strategy):
        self.Genes = genes
        self.Fitness = fitness
        self.Strategy = strategy
        self.Age = 0
 
 
class Strategies(Enum):
    Create = 0,
    Mutate = 1,
    Crossover = 2
 
 
 
class Benchmark:
    @staticmethod
    def run(function):
        timings = []
        stdout = sys.stdout
        for i in range(100):
            sys.stdout = None
            startTime = time.time()
            function()
            seconds = time.time() - startTime
            sys.stdout = stdout
            timings.append(seconds)
            mean = statistics.mean(timings)
            if i < 10 or i % 10 == 9:
                print("{} {:3.2f} {:3.2f}".format(
                    1 + i, mean,statistics.stdev(timings, mean) if i > 1 else 0))

Writing genetic.py


In [ ]:
%%writefile equation.py
import datetime
import random
import unittest
 
import genetic
 
 
def evaluate(genes, prioritizedOperations):
    equation = genes[:]
    for operationSet in prioritizedOperations:
        iOffset = 0
        for i in range(1, len(equation), 2):
            i += iOffset
            opToken = equation[i]
            if opToken in operationSet:
                leftOperand = equation[i - 1]
                rightOperand = equation[i + 1]
                equation[i - 1] = operationSet[opToken](leftOperand,
                                                        rightOperand)
                del equation[i + 1]
                del equation[i]
                iOffset += -2
    return equation[0]
 
 
def add(a, b):
    return a + b
 
 
def subtract(a, b):
    return a - b
 
 
def multiply(a, b):
    return a * b
 
 
def get_fitness(genes, expectedTotal, fnEvaluate):
    result = fnEvaluate(genes)
 
    if result != expectedTotal:
        fitness = expectedTotal - abs(result - expectedTotal)
    else:
        fitness = 1000 - len(genes)
 
    return fitness
 
 
def display(candidate, startTime):
    timeDiff = datetime.datetime.now() - startTime
    print("{}\t{}\t{}".format(
        (' '.join(map(str, [i for i in candidate.Genes]))),
        candidate.Fitness,
        timeDiff))
 
 
def create(numbers, operations, minNumbers, maxNumbers):
    genes = [random.choice(numbers)]
    count = random.randint(minNumbers, 1 + maxNumbers)
    while count > 1:
        count -= 1
        genes.append(random.choice(operations))
        genes.append(random.choice(numbers))
    return genes
 
 
def mutate(genes, numbers, operations, minNumbers, maxNumbers,
           fnGetFitness):
    count = random.randint(1, 10)
    initialFitness = fnGetFitness(genes)
    while count > 0:
        count -= 1
        if fnGetFitness(genes) > initialFitness:
            return
        numberCount = (1 + len(genes)) / 2
        adding = numberCount < maxNumbers and random.randint(0, 100) == 0
        if adding:
            genes.append(random.choice(operations))
            genes.append(random.choice(numbers))
            continue
 
        removing = numberCount > minNumbers and random.randint(0, 20) == 0
        if removing:
            index = random.randrange(0, len(genes) - 1)
            del genes[index]
            del genes[index]
            continue
 
        index = random.randrange(0, len(genes))
        genes[index] = random.choice(operations) \
            if (index & 1) == 1 else random.choice(numbers)
 
 
class EquationGenerationTests(unittest.TestCase):
    def test_addition(self):
        operations = ['+', '-']
        prioritizedOperations = [{'+': add,
                                  '-': subtract}]
        optimalLengthSolution = [7, '+', 7, '+', 7, '+', 7, '+', 7, '-', 6]
        self.solve(operations, prioritizedOperations, optimalLengthSolution)
 
    def test_multiplication(self):
        operations = ['+', '-', '*']
        prioritizedOperations = [{'*': multiply},
                                 {'+': add,
                                  '-': subtract}]
        optimalLengthSolution = [6, '*', 3, '*', 3, '*', 6, '-', 7]
        self.solve(operations, prioritizedOperations, optimalLengthSolution)
 
    def test_exponent(self):
        operations = ['^', '+', '-', '*']
        prioritizedOperations = [{'^': lambda a, b: a ** b},
                                 {'*': multiply},
                                 {'+': add,
                                  '-': subtract}]
        optimalLengthSolution = [6, '^', 3, '*', 2, '-', 5]
        self.solve(operations, prioritizedOperations, optimalLengthSolution)
 
    def solve(self, operations, prioritizedOperations,
              optimalLengthSolution):
        numbers = [1, 2, 3, 4, 5, 6, 7]
        expectedTotal = evaluate(optimalLengthSolution,
                                 prioritizedOperations)
        minNumbers = (1 + len(optimalLengthSolution)) / 2
        maxNumbers = 6 * minNumbers
        startTime = datetime.datetime.now()
 
        def fnDisplay(candidate):
            display(candidate, startTime)
 
        def fnEvaluate(genes):
            return evaluate(genes, prioritizedOperations)
 
        def fnGetFitness(genes):
            return get_fitness(genes, expectedTotal, fnEvaluate)
 
        def fnCreate():
            return create(numbers, operations, minNumbers, maxNumbers)
 
        def fnMutate(child):
            mutate(child, numbers, operations, minNumbers, maxNumbers,
                   fnGetFitness)
 
        optimalFitness = fnGetFitness(optimalLengthSolution)
        best = genetic.get_best(fnGetFitness, None, optimalFitness, None,
                                fnDisplay, fnMutate, fnCreate, maxAge=50)
        self.assertTrue(not optimalFitness > best.Fitness)
 
    # def test_benchmark(self):
    #     genetic.Benchmark.run(self.test_exponent)
 
 
if __name__ == '__main__':
    unittest.main()

Writing equation.py


In [ ]:
%%shell
python equation.py

7 + 4 + 5 + 5 - 7 - 5 + 2 + 4 - 6	9	0:00:00.000067
7 + 6 + 5 + 5 - 7 - 5 + 2 + 4 - 6	11	0:00:00.000163
7 + 6 + 5 + 5 - 7 + 5 + 4 - 4 - 7	14	0:00:00.000251
7 + 6 + 5 + 5 - 7 + 5 + 4 - 4 - 4	17	0:00:00.000297
7 + 6 + 5 + 4 - 7 + 5 + 4 + 4 - 4	24	0:00:00.000349
7 + 6 + 5 + 4 - 5 + 5 + 4 + 4 - 4	26	0:00:00.000391
7 + 6 + 5 + 3 - 7 + 5 + 4 + 4 + 4	27	0:00:00.000513
7 + 5 + 5 + 3 - 7 + 5 + 4 + 4 + 4	28	0:00:00.000606
7 + 5 + 5 + 1 - 6 + 5 + 4 + 4 + 4	983	0:00:00.001125
7 + 5 + 5 + 1 - 4 + 7 + 4 + 4	985	0:00:00.002547
7 + 7 + 5 + 4 + 5 + 1	989	0:00:00.005756
.1 0.22 0.00
2 0.14 0.00
3 0.17 0.11
4 0.21 0.11
5 0.21 0.10
6 0.26 0.17
7 0.25 0.16
8 0.28 0.18
9 0.32 0.20
10 0.31 0.19
20 0.29 0.18
30 0.30 0.19
40 0.31 0.20
50 0.32 0.19
60 0.33 0.19
70 0.34 0.23
80 0.32 0.22
90 0.32 0.22
100 0.31 0.22
.7 ^ 7 - 3 * 7 * 4 - 2 + 5 - 1 * 4 * 2 * 1 - 7 + 7 - 3 - 3 + 7 - 1 - 7 - 7 + 1 ^ 5 ^ 6	-822587	0:00:00.000084
7 ^ 7 - 3 * 7 * 4 - 2 + 5 - 1 * 4 * 2 * 1 - 7 + 5 - 3 - 3 + 7 - 1 - 7 - 7 + 1 ^ 5 ^ 6	-82258

# new start
----

In [ ]:
import random
random.seed(100)

c1=1
c2=2
c3=-3
c4=1
c5=4
c6=1
C=-30
# equation = c1*a+c2*b+c3*c+c4*d+c5*e+c6*f+C
# a+2b-3c+d+4e+f - 30 = 0
def printEquation():
  eq=""
  for var in [(c1,'a'),(c2,'b'),(c3,'c'),(c4,'d'),(c5,'e'),(c6,'f'),(C,'C')]:    
    if(var[0]<0):
      if var[1]=='C':
        eq=eq+str(var[0])
      else:
        eq=eq+str(var[0])+var[1]
    else:
      if var[1]=='C':
        eq=eq+"+"+str(var[0])
      elif var[1]=='a':
        eq=eq+str(var[0])+var[1]
      else:
        eq=eq+"+"+str(var[0])+var[1]
  eq=eq+"=0"
  print("equation : "+eq)

def selection(population, fit_chromosomes, generation):
    fitnessscore=[]
    for chromosome in population:
        individual_fitness=100-abs((c1*chromosome[0]+c2*chromosome[1]+c3*chromosome[2]+c4*chromosome[3]+c5*chromosome[4]+c6*chromosome[5])+C)
        fitnessscore.append(individual_fitness)

    total_fitness=sum(fitnessscore)
    print('Total fitness: ', total_fitness)
    score_card=list(zip(fitnessscore,population))
    score_card.sort(reverse=True)

    for individual in score_card:
        if individual[0]==100:
            if individual[1] not in fit_chromosomes:
                fit_chromosomes.append(individual[1])
    
    score_card=score_card[:4]
    score, population=zip(*score_card)
    return list(population)

def crossover(population):
    random.shuffle(population)
    fatherchromosome=population[:2]
    motherchromosome=population[2:]
    children=[]
    for i in range(len(fatherchromosome)):
        crossoversite=random.randint(1,5)
        fatherfragments=[fatherchromosome[i][:crossoversite],fatherchromosome[i][crossoversite:]]
        motherfragments=[motherchromosome[i][:crossoversite],motherchromosome[i][crossoversite:]]
        firstchild=fatherfragments[0]+motherfragments[1]
        children.append(firstchild)
        secondchild=motherfragments[0]+fatherfragments[1]
        children.append(secondchild)
    return children

def mutation(population):
    mutatedchromosomes=[]
    for chromosome in population:
        mutation_site=random.randint(0,5)
        chromosome[mutation_site]=random.randint(1,9)
        mutatedchromosomes.append(chromosome)
    return mutatedchromosomes

def get_fit_chromosomes(generations):
    population=[[random.randint(1,9) for i in range(6)] for j in range(6)]
    fit_chromosomes=[]
    for generation in range(generations):
        generation+=1
        print('Generation:', generation)
        population=selection(population, fit_chromosomes, generation)
        crossover_children=crossover(population)
        population=population+crossover_children
        mutated_population=mutation(population)
        population=population+mutated_population
        #random.shuffle(population)

    return fit_chromosomes

def get_coefficient():
  global c1
  global c2
  global c3
  global c4
  global c5
  global c6
  global C
  print("Equation : (c1 * a) + (c2 * b) + (c3 * c) + (c4 * d) + (c5 * e) + (c6 * f) + C * Constant = 0 ")
  c1 = int(input('Enter coefficient of a:'))
  c2 = int(input('Enter coefficient of b:'))
  c3 = int(input('Enter coefficient of c:'))
  c4 = int(input('Enter coefficient of d:'))
  c5 = int(input('Enter coefficient of e:'))
  c6 = int(input('Enter coefficient of f:'))
  C = int(input('Enter constant:'))
  printEquation()

# get_coefficient()
solution=get_fit_chromosomes(20)
print('-----------Solution-----------')
print(solution)
print(len(solution))
printEquation()

Generation: 1
Total fitness:  541
Generation: 2
Total fitness:  1502
Generation: 3
Total fitness:  1432
Generation: 4
Total fitness:  1422
Generation: 5
Total fitness:  1424
Generation: 6
Total fitness:  1480
Generation: 7
Total fitness:  1480
Generation: 8
Total fitness:  1456
Generation: 9
Total fitness:  1478
Generation: 10
Total fitness:  1480
Generation: 11
Total fitness:  1454
Generation: 12
Total fitness:  1528
Generation: 13
Total fitness:  1544
Generation: 14
Total fitness:  1380
Generation: 15
Total fitness:  1490
Generation: 16
Total fitness:  1434
Generation: 17
Total fitness:  1480
Generation: 18
Total fitness:  1524
Generation: 19
Total fitness:  1524
Generation: 20
Total fitness:  1418
-----------Solution-----------
[[5, 7, 8, 7, 3, 6], [7, 9, 8, 8, 7, 6], [7, 6, 8, 5, 3, 6], [4, 4, 8, 4, 7, 7], [1, 7, 7, 6, 2, 1], [5, 7, 3, 9, 7, 1]]
6
equation : 1a+2b-3c+1d+4e+1f-30=0
